# HPO: Hyper-parameter Optimization

[HPO Method]
1. Grid Search
2. Random Search
3. Bayesian Search
4. Bayesian Search(TPE:Tree-structured Parzen Estimator) using Optuna

[Data]  
Bank Marketing Data set  
https://archive.ics.uci.edu/dataset/222/bank+marketing

[Note]  
HPO Example로 오류방지를 위한 최소한의 전처리만 수행


[Task 1]  
위 예시 사례를 활용하고 적용모델을 달리하여 하이퍼 파라메타 최적화(HPO)를 수행해보세요  
 - HPO Method 중 2개의 Method를 선정하여 비교분석 해보세요.
 - 적용 모델을 다르게.. (XGBoost 제외)
  
   
[Task 2]  
하기 데이터를 활용하여 하이퍼 파라메타 최적화(HPO)를 수행해보세요  
 - HPO Method 중 2개의 Method를 선정하여 비교분석 해보세요.
 - 모델선택은 자유  
  
활용 데이터 : Adult(Census Income)(2)    
https://archive.ics.uci.edu/dataset/2/adult


In [17]:
#  UCI Machine Learning Data Load Library
# !pip install ucimlrepo

In [18]:
#  Bayesian Search Library
# !pip install scikit-optimize

In [19]:
#  Bayesian Search(TPE) Library
# !pip install optuna

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, LeaveOneOut
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import time
import xgboost as xgb


## 1. Data Load

In [5]:
from ucimlrepo import fetch_ucirepo

def load_data():

    # fetch dataset
    bank_marketing = fetch_ucirepo(id=222)

    # data (as pandas dataframes)
    X = bank_marketing.data.features
    y = bank_marketing.data.targets

    # Concatenate
    df = pd.concat([X, y], axis=1)

    print(f"Shape: {df.shape}")
    print(f"Info: {df.info()}")
    print(f"Samples: {df.head(5)}")
    print("Bank Marketing Dataset load")

    return df, X, y

# 1.데이터 로드
df, X, y = load_data()

Shape: (45211, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          45211 non-null  int64 
 1   job          44923 non-null  object
 2   marital      45211 non-null  object
 3   education    43354 non-null  object
 4   default      45211 non-null  object
 5   balance      45211 non-null  int64 
 6   housing      45211 non-null  object
 7   loan         45211 non-null  object
 8   contact      32191 non-null  object
 9   day_of_week  45211 non-null  int64 
 10  month        45211 non-null  object
 11  duration     45211 non-null  int64 
 12  campaign     45211 non-null  int64 
 13  pdays        45211 non-null  int64 
 14  previous     45211 non-null  int64 
 15  poutcome     8252 non-null   object
 16  y            45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB
Info: None
Samples:    age           job  

## 2. EDA

In [6]:
def eda_data(df):

    # 기본 통계
    print("데이터 타입:")
    print(df.dtypes)

    print("\n결측치 확인:")
    print(df.isnull().sum())

    print("\n타겟 변수 분포:")
    target_counts = df['y'].value_counts()
    print(target_counts)
    print(f"비율: {target_counts / len(df) * 100}")

    print("\n숫자형 변수 기본 통계:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print(df[numeric_cols].describe())

    print("\n범주형 변수 고유값 개수:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        print(f"{col}: {df[col].nunique()}개 고유값")
        if df[col].nunique() <= 10:  # 고유값이 적으면 분포 출력
            print(df[col].value_counts().head())
        print()

# 2.데이터 탐색
eda_data(df)

데이터 타입:
age             int64
job            object
marital        object
education      object
default        object
balance         int64
housing        object
loan           object
contact        object
day_of_week     int64
month          object
duration        int64
campaign        int64
pdays           int64
previous        int64
poutcome       object
y              object
dtype: object

결측치 확인:
age                0
job              288
marital            0
education       1857
default            0
balance            0
housing            0
loan               0
contact        13020
day_of_week        0
month              0
duration           0
campaign           0
pdays              0
previous           0
poutcome       36959
y                  0
dtype: int64

타겟 변수 분포:
y
no     39922
yes     5289
Name: count, dtype: int64
비율: y
no     88.30152
yes    11.69848
Name: count, dtype: float64

숫자형 변수 기본 통계:
                age        balance   day_of_week      duration      campaign  \

## 3. Pre-processing

In [7]:
def preprocess_data(X, y):

    # 데이터프레임 복사
    X_processed = X.copy()

    # 범주형 변수들을 레이블 인코딩
    categorical_columns = X_processed.select_dtypes(include=['object']).columns
    label_encoders = {}

    for col in categorical_columns:
        le = LabelEncoder()
        X_processed[col] = le.fit_transform(X_processed[col].astype(str))
        label_encoders[col] = le
        print(f"{col} 인코딩 완료: {len(le.classes_)}개 클래스")

    # 타겟 변수 인코딩 (yes=1, no=0)
    y_processed = (y['y'] == 'yes').astype(int)

    print(f"타겟 분포: {y_processed.value_counts().to_dict()}")

    return X_processed, y_processed, label_encoders


# 3. 데이터 전처리
X_processed, y_processed, label_encoders = preprocess_data(X, y)

job 인코딩 완료: 12개 클래스
marital 인코딩 완료: 3개 클래스
education 인코딩 완료: 4개 클래스
default 인코딩 완료: 2개 클래스
housing 인코딩 완료: 2개 클래스
loan 인코딩 완료: 2개 클래스
contact 인코딩 완료: 3개 클래스
month 인코딩 완료: 12개 클래스
poutcome 인코딩 완료: 4개 클래스
타겟 분포: {0: 39922, 1: 5289}


In [8]:
label_encoders

{'job': LabelEncoder(),
 'marital': LabelEncoder(),
 'education': LabelEncoder(),
 'default': LabelEncoder(),
 'housing': LabelEncoder(),
 'loan': LabelEncoder(),
 'contact': LabelEncoder(),
 'month': LabelEncoder(),
 'poutcome': LabelEncoder()}

## 4. Baseline Modeling & Evaluation

In [9]:
def build_and_evaluate_model(X, y):
    """XGBoost 모델 구축 및 평가"""
    print("\n=== XGBoost 모델 구축 및 평가 ===")

    # 훈련/테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"훈련 데이터: {X_train.shape}")
    print(f"테스트 데이터: {X_test.shape}")

    # 특성 스케일링
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # XGBoost 모델 훈련
    xgb_model = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        early_stopping_rounds=5,
        eval_metric='logloss'
    )

    print("XGBoost Learning...")
    xgb_model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_test_scaled, y_test)],
        verbose=False
    )

    # 예측
    y_pred = xgb_model.predict(X_test_scaled)
    y_pred_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

    # 평가
    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_pred_proba)

    print(f"\n정확도: {accuracy:.4f}")
    print(f"AUC 점수: {auc_score:.4f}")

    print("\n분류 리포트:")
    print(classification_report(y_test, y_pred))

    print("\n혼동 행렬:")
    print(confusion_matrix(y_test, y_pred))

    # 특성 중요도
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\n상위 10개 중요한 특성:")
    print(feature_importance.head(10))

    return xgb_model, scaler, feature_importance

# 4. 모델 구축 및 평가
model, scaler, feature_importance = build_and_evaluate_model(X_processed, y_processed)



=== XGBoost 모델 구축 및 평가 ===
훈련 데이터: (36168, 16)
테스트 데이터: (9043, 16)
XGBoost Learning...

정확도: 0.9103
AUC 점수: 0.9314

분류 리포트:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      7985
           1       0.67      0.45      0.54      1058

    accuracy                           0.91      9043
   macro avg       0.80      0.71      0.75      9043
weighted avg       0.90      0.91      0.90      9043


혼동 행렬:
[[7752  233]
 [ 578  480]]

상위 10개 중요한 특성:
        feature  importance
15     poutcome    0.333886
11     duration    0.180560
6       housing    0.101092
8       contact    0.088195
10        month    0.055407
7          loan    0.037297
13        pdays    0.031768
0           age    0.026959
12     campaign    0.023817
9   day_of_week    0.023566


## 5. Cross-Validation Method

In [10]:
def compare_cv_strategies(X, y):
    """다양한 CV 전략 비교"""
    from sklearn.model_selection import KFold, RepeatedStratifiedKFold

    print("\n=== Cross-Validation 전략 비교 ===")

    # 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 모델
    xgb_model = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        eval_metric='logloss'
    )

    # CV 전략들
    cv_strategies = {
    #    'Leave-One-Out (LOOCV)': LeaveOneOut(),
        'KFold (5-fold)': KFold(n_splits=5, shuffle=True, random_state=42),
        'StratifiedKFold (5-fold)': StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        'StratifiedKFold (20-fold)': StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
    }

    results = {}
    for name, cv_strategy in cv_strategies.items():
        scores = cross_val_score(xgb_model, X_scaled, y, cv=cv_strategy, scoring='roc_auc', n_jobs=-1)
        results[name] = {
            'mean': scores.mean(),
            'std': scores.std(),
            'scores': scores
        }
        print(f"{name}: {scores.mean():.4f} (±{scores.std()*2:.4f})")

    return results


results = compare_cv_strategies(X_processed, y_processed)


=== Cross-Validation 전략 비교 ===


KFold (5-fold): 0.9330 (±0.0046)
StratifiedKFold (5-fold): 0.9333 (±0.0083)
StratifiedKFold (20-fold): 0.9341 (±0.0104)


## 6-1. HPO: Grid Search

In [11]:
def grid_search_tuning_cv(X, y):
    """Cross-Validation을 활용한 그리드 서치 하이퍼파라미터 튜닝"""
    from sklearn.model_selection import GridSearchCV
    print("\n=== 하이퍼파라미터 튜닝 (GridSearchCV) ===")
    t0 = time.perf_counter()

    # 파라미터 그리드 정의
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [4, 6, 8],
        'learning_rate': [0.05, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    }

    # XGBoost 모델
    xgb_model = xgb.XGBClassifier(
        objective='binary:logistic',
        random_state=42,
        eval_metric='logloss'
    )

    # GridSearchCV 설정
    grid_search = GridSearchCV(
        estimator=xgb_model,     # 최적화를 적용할 모델
        param_grid=param_grid,   # 탐색할 하이퍼파라미터 공간
        scoring='roc_auc',
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,               # 병렬 실행 스레드 수, -1 = CPU 전체 사용
        verbose=1
    )

    # 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 그리드 서치 실행
    print("GridSearchCV 실행 중...")
    grid_search.fit(X_scaled, y)

    dt = time.perf_counter() - t0
    print(f"수행 시간: {dt:.3f}s")
    print(f"최적 파라미터: {grid_search.best_params_}")
    print(f"최적 CV 점수: {grid_search.best_score_:.4f}")


    return grid_search.best_estimator_, scaler


best_estimator_, scaler = grid_search_tuning_cv(X_processed, y_processed)



=== 하이퍼파라미터 튜닝 (GridSearchCV) ===
GridSearchCV 실행 중...
Fitting 5 folds for each of 81 candidates, totalling 405 fits
수행 시간: 12.976s
최적 파라미터: {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200, 'subsample': 0.8}
최적 CV 점수: 0.9350


## 6-2. HPO: Random Search

In [12]:
def random_search_tuning(X, y):
    """Cross-Validation을 활용한 랜덤 서치 하이퍼파라미터 튜닝"""
    from sklearn.model_selection import RandomizedSearchCV
    from scipy.stats import uniform, randint

    t0 = time.perf_counter()

    print("\n=== 하이퍼파라미터 튜닝 (RandomizedSearchCV) ===")

    # 랜덤 서치를 위한 파라미터 분포 정의
    param_distributions = {
        'n_estimators': [50, 100, 200],
        'max_depth': [4, 6, 8],
        'learning_rate': [0.05, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
      #  'colsample_bytree': uniform(0.6, 0.4),
    }

    # XGBoost 모델
    xgb_model = xgb.XGBClassifier(
        objective='binary:logistic',
        random_state=42,
        eval_metric='logloss'
    )

    # RandomizedSearchCV 설정
    random_search = RandomizedSearchCV(
        estimator=xgb_model,      # 최적화를 적용할 모델
        param_distributions=param_distributions,  # 탐색할 하이퍼파라미터 분포
        n_iter=50,               # 탐색할 반복 횟수
        scoring='roc_auc',
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,                # 병렬 실행 스레드 수, -1 = CPU 전체 사용
        verbose=1,
        random_state=42
    )

    # 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 랜덤 서치 실행
    print("RandomizedSearchCV 실행 중...")
    random_search.fit(X_scaled, y)

    dt = time.perf_counter() - t0
    print(f"수행 시간: {dt:.3f}s")
    print(f"최적 파라미터: {random_search.best_params_}")
    print(f"최적 CV 점수: {random_search.best_score_:.4f}")

    # 상위 5개 결과 출력
    print("\n상위 5개 파라미터 조합:")
    results_df = pd.DataFrame(random_search.cv_results_)
    top_5 = results_df.nlargest(5, 'mean_test_score')[['mean_test_score', 'std_test_score', 'params']]

    for idx, (_, row) in enumerate(top_5.iterrows(), 1):
        print(f"{idx}. 점수: {row['mean_test_score']:.4f} (±{row['std_test_score']*2:.4f})")
        print(f"   파라미터: {row['params']}")

    return random_search.best_estimator_, scaler, random_search

best_estimator_, scaler, random_search = random_search_tuning(X_processed, y_processed)



=== 하이퍼파라미터 튜닝 (RandomizedSearchCV) ===
RandomizedSearchCV 실행 중...
Fitting 5 folds for each of 50 candidates, totalling 250 fits
수행 시간: 7.856s
최적 파라미터: {'subsample': 0.9, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.05}
최적 CV 점수: 0.9348

상위 5개 파라미터 조합:
1. 점수: 0.9348 (±0.0075)
   파라미터: {'subsample': 0.9, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.05}
2. 점수: 0.9347 (±0.0072)
   파라미터: {'subsample': 0.9, 'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.05}
3. 점수: 0.9341 (±0.0068)
   파라미터: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1}
4. 점수: 0.9340 (±0.0085)
   파라미터: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1}
5. 점수: 0.9340 (±0.0081)
   파라미터: {'subsample': 0.9, 'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1}


## 6-3. HPO: Bayesian Search

In [13]:
def bayesian_search_tuning(X, y):
    """베이지안 최적화를 활용한 하이퍼파라미터 튜닝"""
    t0 = time.perf_counter()

    from skopt import BayesSearchCV
    from skopt.space import Real, Integer, Categorical

    print("\n=== 하이퍼파라미터 튜닝 (Bayesian Optimization) ===")

    # 베이지안 최적화를 위한 검색 공간 정의
    # log-uniform 분포: 작은 값 쪽에 더 많은 샘플을 할당 → 학습률은 보통 작은 값이 성능에 큰 영향을 주기 때문에 로그 분포로 탐색하는 게 효율적
    search_space = {
        'n_estimators': Integer(50, 200),
        'max_depth': Integer(4, 8),
        'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
        'subsample': Real(0.8, 1.0)
    }


    # XGBoost 모델
    xgb_model = xgb.XGBClassifier(
        objective='binary:logistic',
        random_state=42,
        eval_metric='logloss'
    )

    # BayesSearchCV 설정
    bayes_search = BayesSearchCV(
        estimator=xgb_model,          # 최적화를 적용할 모델
        search_spaces=search_space,   # 탐색할 하이퍼파라미터 공간 (딕셔너리 형식)
        n_iter=30,                    # 탐색할 반복 횟수(기본값:50), 클수록 더 좋은 해를 찾을 확률이 높지만 시간이 오래 걸림, 30~100
        scoring='roc_auc',            # 평가지표
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,                    # 병렬 실행 스레드 수, -1 = CPU 전체 사용
        random_state=42,
        verbose=0                     # 출력 로그 레벨(0 = 없음, 1 이상 = 진행 상황 출력)
    )

    # 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 베이지안 최적화 실행
    print("Bayesian Optimization 실행 중...")
    bayes_search.fit(X_scaled, y)

    dt = time.perf_counter() - t0
    print(f"수행 시간: {dt:.3f}s")

    print(f"최적 파라미터: {bayes_search.best_params_}")
    print(f"최적 CV 점수: {bayes_search.best_score_:.4f}")

    # 최적화 과정 분석
    print(f"\n최적화 과정 분석:")
    print(f"전체 시도 횟수: {len(bayes_search.cv_results_['mean_test_score'])}")

    # 상위 5개 결과 출력
    print("\n상위 5개 파라미터 조합:")
    results_df = pd.DataFrame(bayes_search.cv_results_)
    top_5 = results_df.nlargest(5, 'mean_test_score')[['mean_test_score', 'std_test_score', 'params']]

    for idx, (_, row) in enumerate(top_5.iterrows(), 1):
        print(f"{idx}. 점수: {row['mean_test_score']:.4f} (±{row['std_test_score']*2:.4f})")
        print(f"   파라미터: {row['params']}")

    # 수렴 곡선 데이터 생성
    scores_over_time = []
    best_score_so_far = -np.inf
    for score in bayes_search.cv_results_['mean_test_score']:
        if score > best_score_so_far:
            best_score_so_far = score
        scores_over_time.append(best_score_so_far)

    print(f"\n수렴 분석:")
    print(f"초기 10회 평균 점수: {np.mean(scores_over_time[:10]):.4f}")
    print(f"마지막 10회 최고 점수: {scores_over_time[-1]:.4f}")
    print(f"개선 정도: {(scores_over_time[-1] - np.mean(scores_over_time[:10])):.4f}")

    return bayes_search.best_estimator_, scaler, bayes_search

best_estimator_, scaler, bayes_search = bayesian_search_tuning(X_processed, y_processed)



=== 하이퍼파라미터 튜닝 (Bayesian Optimization) ===
Bayesian Optimization 실행 중...
수행 시간: 26.248s
최적 파라미터: OrderedDict([('learning_rate', 0.07064596568969471), ('max_depth', 6), ('n_estimators', 200), ('subsample', 0.8)])
최적 CV 점수: 0.9349

최적화 과정 분석:
전체 시도 횟수: 30

상위 5개 파라미터 조합:
1. 점수: 0.9349 (±0.0075)
   파라미터: OrderedDict([('learning_rate', 0.07064596568969471), ('max_depth', 6), ('n_estimators', 200), ('subsample', 0.8)])
2. 점수: 0.9348 (±0.0076)
   파라미터: OrderedDict([('learning_rate', 0.08313363930334089), ('max_depth', 6), ('n_estimators', 166), ('subsample', 0.8)])
3. 점수: 0.9345 (±0.0077)
   파라미터: OrderedDict([('learning_rate', 0.07283408237913369), ('max_depth', 8), ('n_estimators', 149), ('subsample', 0.8)])
4. 점수: 0.9342 (±0.0080)
   파라미터: OrderedDict([('learning_rate', 0.040343472274915984), ('max_depth', 7), ('n_estimators', 190), ('subsample', 0.8631599186974098)])
5. 점수: 0.9342 (±0.0069)
   파라미터: OrderedDict([('learning_rate', 0.15849201309311142), ('max_depth', 5), ('n_estimators', 

## 6-4. HPO: Bayesian Search(TPE:Tree-structured Parzen Estimator) using Optuna

In [14]:
import optuna

def optuna_search_tuning(X, y, n_trials=100):
    """Optuna를 활용한 하이퍼파라미터 튜닝"""
    t0 = time.perf_counter()

    print("\n=== 하이퍼파라미터 튜닝 (Optuna) ===")

    # 전역 변수로 데이터를 저장 (objective 함수에서 사용하기 위해)
    global X_train_global, X_val_global, y_train_global, y_val_global, scaler_global

    # 데이터 분할 및 스케일링
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # 전역 변수에 할당
    X_train_global, X_val_global = X_train_scaled, X_val_scaled
    y_train_global, y_val_global = y_train, y_val
    scaler_global = scaler

    def objective(trial):
        """Optuna objective 함수"""

        # 하이퍼파라미터 제안
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'random_state': 42,
            'verbosity': 0,

            # 튜닝할 하이퍼파라미터들
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'max_depth': trial.suggest_int('max_depth', 4, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.8, 1.0)
               }

        # XGBoost 모델 생성 및 훈련
        model = xgb.XGBClassifier(**params)

        model.fit(
            X_train_global, y_train_global,
            eval_set=[(X_val_global, y_val_global)],
            verbose=False
        )

        # 검증 성능 계산
        y_pred_proba = model.predict_proba(X_val_global)[:, 1]
        auc_score = roc_auc_score(y_val_global, y_pred_proba)

        return auc_score

    # Optuna 스터디 생성
    print(f"Optuna 최적화 시작 ({n_trials}회 시도)...")

    # 로그 출력 줄이기
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    study = optuna.create_study(
        direction='maximize',
        sampler=optuna.samplers.TPESampler(seed=42),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    )

    # 최적화 실행
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    # 결과 출력
    dt = time.perf_counter() - t0
    print(f"수행 시간: {dt:.3f}s")
    print(f"최적 AUC 점수: {study.best_value:.4f}")
    print(f"최적 파라미터: {study.best_params}")
    print(f"시도 횟수: {len(study.trials)}")

    # 최적 모델 재훈련
    best_params = study.best_params.copy()
    best_params.update({
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'random_state': 42
    })

    best_model = xgb.XGBClassifier(**best_params)
    best_model.fit(X_train_global, y_train_global)

    # 최적화 과정 분석
    print(f"\n=== 최적화 과정 분석 ===")

    # 상위 5개 시도 결과
    best_trials = sorted(study.trials, key=lambda t: t.value if t.value else -np.inf, reverse=True)[:5]
    print("상위 5개 시도 결과:")
    for i, trial in enumerate(best_trials, 1):
        if trial.value:
            print(f"{i}. AUC: {trial.value:.4f}, 파라미터: {trial.params}")

    # 수렴 분석
    values = [trial.value for trial in study.trials if trial.value is not None]
    if len(values) >= 10:
        early_avg = np.mean(values[:len(values)//2])
        late_avg = np.mean(values[len(values)//2:])
        print(f"\n수렴 분석:")
        print(f"전반부 평균 AUC: {early_avg:.4f}")
        print(f"후반부 평균 AUC: {late_avg:.4f}")
        print(f"개선 정도: {late_avg - early_avg:.4f}")

    # 파라미터 중요도 분석
    importance = optuna.importance.get_param_importances(study)
    print(f"\n파라미터 중요도 (Top 5):")
    for param, imp in sorted(importance.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"  {param}: {imp:.4f}")

    return best_model, scaler_global, study

best_model, scaler_global, study = optuna_search_tuning(X_processed, y_processed, n_trials=30)


=== 하이퍼파라미터 튜닝 (Optuna) ===
Optuna 최적화 시작 (30회 시도)...


  0%|          | 0/30 [00:00<?, ?it/s]

수행 시간: 7.245s
최적 AUC 점수: 0.9323
최적 파라미터: {'n_estimators': 196, 'max_depth': 5, 'learning_rate': 0.053566770906231354, 'subsample': 0.8017372221127864}
시도 횟수: 30

=== 최적화 과정 분석 ===
상위 5개 시도 결과:
1. AUC: 0.9323, 파라미터: {'n_estimators': 196, 'max_depth': 5, 'learning_rate': 0.053566770906231354, 'subsample': 0.8017372221127864}
2. AUC: 0.9323, 파라미터: {'n_estimators': 195, 'max_depth': 5, 'learning_rate': 0.07704669005072712, 'subsample': 0.802358795884076}
3. AUC: 0.9323, 파라미터: {'n_estimators': 185, 'max_depth': 6, 'learning_rate': 0.054138800373132734, 'subsample': 0.8191863327780846}
4. AUC: 0.9323, 파라미터: {'n_estimators': 169, 'max_depth': 6, 'learning_rate': 0.0957534158847779, 'subsample': 0.8242648065638125}
5. AUC: 0.9323, 파라미터: {'n_estimators': 182, 'max_depth': 6, 'learning_rate': 0.052102913140168655, 'subsample': 0.8742760522111468}

수렴 분석:
전반부 평균 AUC: 0.9237
후반부 평균 AUC: 0.9295
개선 정도: 0.0058

파라미터 중요도 (Top 5):
  learning_rate: 0.5394
  n_estimators: 0.2261
  subsample: 0.1944
  max

## Task1: 위 예시 사례를 활용하고 적용모델을 달리하여 하이퍼 파라메타 최적화(HPO) 수행  
 - HPO Method 중 2개의 Method를 선정하여 비교분석 해보세요.
 - 적용 모델을 다르게.. (XGBoost 제외)

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

print("=== Task 1: 다른 모델을 활용한 HPO 비교 분석 ===")
print("모델: Random Forest vs SVM")
print("HPO 방법: Grid Search vs Bayesian Optimization")

# 데이터 준비
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, test_size=0.2, random_state=42, stratify=y_processed
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n훈련 데이터: {X_train_scaled.shape}")
print(f"테스트 데이터: {X_test_scaled.shape}")

# === 1. Random Forest with Grid Search ===
def rf_grid_search(X_train, y_train):
    print("\n=== Random Forest + Grid Search ===")
    t0 = time.perf_counter()
    
    # Random Forest 파라미터 그리드
    rf_param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    }
    
    rf_model = RandomForestClassifier(random_state=42)
    
    rf_grid_search = GridSearchCV(
        estimator=rf_model,
        param_grid=rf_param_grid,
        scoring='roc_auc',
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,
        verbose=1
    )
    
    rf_grid_search.fit(X_train, y_train)
    
    dt = time.perf_counter() - t0
    print(f"수행 시간: {dt:.3f}s")
    print(f"최적 파라미터: {rf_grid_search.best_params_}")
    print(f"최적 CV 점수: {rf_grid_search.best_score_:.4f}")
    
    return rf_grid_search.best_estimator_, dt

# === 2. Random Forest with Bayesian Optimization ===
def rf_bayesian_search(X_train, y_train):
    print("\n=== Random Forest + Bayesian Optimization ===")
    t0 = time.perf_counter()
    
    # Random Forest 검색 공간
    rf_search_space = {
        'n_estimators': Integer(50, 200),
        'max_depth': Integer(5, 20),
        'min_samples_split': Integer(2, 20),
        'min_samples_leaf': Integer(1, 10),
        'max_features': Categorical(['sqrt', 'log2'])
    }
    
    rf_model = RandomForestClassifier(random_state=42)
    
    rf_bayes_search = BayesSearchCV(
        estimator=rf_model,
        search_spaces=rf_search_space,
        n_iter=30,
        scoring='roc_auc',
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    
    rf_bayes_search.fit(X_train, y_train)
    
    dt = time.perf_counter() - t0
    print(f"수행 시간: {dt:.3f}s")
    print(f"최적 파라미터: {rf_bayes_search.best_params_}")
    print(f"최적 CV 점수: {rf_bayes_search.best_score_:.4f}")
    
    return rf_bayes_search.best_estimator_, dt

# === 3. SVM with Grid Search ===
def svm_grid_search(X_train, y_train):
    print("\n=== SVM + Grid Search ===")
    t0 = time.perf_counter()
    
    # SVM 파라미터 그리드 (작은 그리드로 시간 단축)
    svm_param_grid = {
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto', 0.001, 0.01],
        'kernel': ['rbf', 'linear']
    }
    
    svm_model = SVC(random_state=42, probability=True)
    
    svm_grid_search = GridSearchCV(
        estimator=svm_model,
        param_grid=svm_param_grid,
        scoring='roc_auc',
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),  # 3-fold로 시간 단축
        n_jobs=-1,
        verbose=1
    )
    
    svm_grid_search.fit(X_train, y_train)
    
    dt = time.perf_counter() - t0
    print(f"수행 시간: {dt:.3f}s")
    print(f"최적 파라미터: {svm_grid_search.best_params_}")
    print(f"최적 CV 점수: {svm_grid_search.best_score_:.4f}")
    
    return svm_grid_search.best_estimator_, dt

# === 4. SVM with Bayesian Optimization ===
def svm_bayesian_search(X_train, y_train):
    print("\n=== SVM + Bayesian Optimization ===")
    t0 = time.perf_counter()
    
    # SVM 검색 공간
    svm_search_space = {
        'C': Real(0.1, 100, prior='log-uniform'),
        'gamma': Real(1e-4, 1e-1, prior='log-uniform'),
        'kernel': Categorical(['rbf', 'linear'])
    }
    
    svm_model = SVC(random_state=42, probability=True)
    
    svm_bayes_search = BayesSearchCV(
        estimator=svm_model,
        search_spaces=svm_search_space,
        n_iter=20,
        scoring='roc_auc',
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),  # 3-fold로 시간 단축
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    
    svm_bayes_search.fit(X_train, y_train)
    
    dt = time.perf_counter() - t0
    print(f"수행 시간: {dt:.3f}s")
    print(f"최적 파라미터: {svm_bayes_search.best_params_}")
    print(f"최적 CV 점수: {svm_bayes_search.best_score_:.4f}")
    
    return svm_bayes_search.best_estimator_, dt

# === 모든 방법 실행 및 비교 ===
results = {}

# Random Forest 실행
rf_grid_model, rf_grid_time = rf_grid_search(X_train_scaled, y_train)
rf_bayes_model, rf_bayes_time = rf_bayesian_search(X_train_scaled, y_train)

# SVM 실행
svm_grid_model, svm_grid_time = svm_grid_search(X_train_scaled, y_train)
svm_bayes_model, svm_bayes_time = svm_bayesian_search(X_train_scaled, y_train)

# === 테스트 성능 평가 ===
models = {
    'RF + Grid Search': rf_grid_model,
    'RF + Bayesian': rf_bayes_model,
    'SVM + Grid Search': svm_grid_model,
    'SVM + Bayesian': svm_bayes_model
}

times = {
    'RF + Grid Search': rf_grid_time,
    'RF + Bayesian': rf_bayes_time,
    'SVM + Grid Search': svm_grid_time,
    'SVM + Bayesian': svm_bayes_time
}

print("\n" + "="*60)
print("=== 최종 테스트 성능 비교 ===")
print("="*60)

test_results = []

for name, model in models.items():
    # 예측
    if name.startswith('SVM'):
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:  # Random Forest
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 성능 계산
    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    test_results.append({
        'Method': name,
        'Accuracy': accuracy,
        'AUC': auc_score,
        'Time(s)': times[name]
    })
    
    print(f"\n{name}:")
    print(f"  정확도: {accuracy:.4f}")
    print(f"  AUC: {auc_score:.4f}")
    print(f"  최적화 시간: {times[name]:.3f}s")

# 결과 DataFrame 생성
results_df = pd.DataFrame(test_results)
print(f"\n=== 종합 비교 결과 ===")
print(results_df.round(4))

# 최고 성능 모델 찾기
best_auc_idx = results_df['AUC'].idxmax()
best_accuracy_idx = results_df['Accuracy'].idxmax()
fastest_idx = results_df['Time(s)'].idxmin()

print(f"\n=== 요약 ===")
print(f"최고 AUC: {results_df.loc[best_auc_idx, 'Method']} ({results_df.loc[best_auc_idx, 'AUC']:.4f})")
print(f"최고 정확도: {results_df.loc[best_accuracy_idx, 'Method']} ({results_df.loc[best_accuracy_idx, 'Accuracy']:.4f})")
print(f"가장 빠른 최적화: {results_df.loc[fastest_idx, 'Method']} ({results_df.loc[fastest_idx, 'Time(s)']:.3f}s)")

# 시각화
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# AUC 비교
axes[0].bar(results_df['Method'], results_df['AUC'])
axes[0].set_title('AUC Score Comparison')
axes[0].set_ylabel('AUC Score')
axes[0].tick_params(axis='x', rotation=45)

# 정확도 비교
axes[1].bar(results_df['Method'], results_df['Accuracy'])
axes[1].set_title('Accuracy Comparison')
axes[1].set_ylabel('Accuracy')
axes[1].tick_params(axis='x', rotation=45)

# 시간 비교
axes[2].bar(results_df['Method'], results_df['Time(s)'])
axes[2].set_title('Optimization Time Comparison')
axes[2].set_ylabel('Time (seconds)')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

=== Task 1: 다른 모델을 활용한 HPO 비교 분석 ===
모델: Random Forest vs SVM
HPO 방법: Grid Search vs Bayesian Optimization

훈련 데이터: (36168, 16)
테스트 데이터: (9043, 16)

=== Random Forest + Grid Search ===
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
수행 시간: 323.300s
최적 파라미터: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
최적 CV 점수: 0.9286

=== Random Forest + Bayesian Optimization ===
수행 시간: 130.856s
최적 파라미터: OrderedDict([('max_depth', 20), ('max_features', 'sqrt'), ('min_samples_leaf', 1), ('min_samples_split', 18), ('n_estimators', 196)])
최적 CV 점수: 0.9285

=== SVM + Grid Search ===
Fitting 3 folds for each of 24 candidates, totalling 72 fits
수행 시간: 1117.618s
최적 파라미터: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
최적 CV 점수: 0.8892

=== SVM + Bayesian Optimization ===


## Task2: 다른 데이터셋 활용하여 하이퍼 파라메타 최적화(HPO) 비교 수행  

- HPO Method 중 2개의 Method를 선정하여 비교분석 해보세요.
- 모델선택은 자유
- 활용 데이터 : Adult(Census Income)(2)
- https://archive.ics.uci.edu/dataset/2/adult

In [16]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from ucimlrepo import fetch_ucirepo
import pandas as pd

# UCI에서 데이터 불러오기
adult = fetch_ucirepo(id=2)

X = adult.data.features
y = adult.data.targets

# Concatenate후 살펴보기
df = pd.concat([X, y], axis=1)

print(f"Data Shape: {df.shape}")
print(f"Categorical Variable: {X.select_dtypes(include=['object']).columns.tolist()}")
print(f"Info: {df.info()}")

print(f"{df.head(5)}")

Data Shape: (48842, 15)
Categorical Variable: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-nu